In [4]:
import logging
import os
import math

import numpy as np

import torch
from torch.utils.data import Dataset

logger = logging.getLogger(__name__)
from typing import Tuple

In [1]:
import sys
sys.path.append('C:\\Users\\NGN\\dev\\Traffino\\TRAFFINO')
from traffino.data.loader import data_loader
path = 'C:\\Users\\NGN\\dev\\Traffino\\TRAFFINO\\traffino\\datasets\\waterloo\\train'
train_dset, train_loader = data_loader(args, path)


NameError: name 'args' is not defined

In [5]:
def read_file(_path, delim='\t'):
    data = []
    if delim == 'tab':
        delim = '\t'
    elif delim == 'space':
        delim = ' '
    with open(_path, 'r') as f:
        for line in f:
            line = line.strip().split(delim)
            line = [float(i) for i in line]
            data.append(line)
    return np.asarray(data)

In [40]:
path = 'C:\\Users\\NGN\\dev\\Traffino\\TRAFFINO\\traffino\\datasets\\waterloo\\train\\0769_prep.txt'
path2 = 'C:\\Users\\NGN\\dev\\Traffino\\TRAFFINO\\traffino\\datasets\\waterloo\\train\\0769_prep2.txt'
path3 = 'C:\\Users\\NGN\\dev\\Traffino\\TRAFFINO\\traffino\\datasets\\waterloo\\train\\0769_prep3.txt'
delim = '\t'
data = read_file(path, delim)
data2 = read_file(path2, delim)
data3 = read_file(path3, delim)
print(len(data))

16387


In [41]:
frames = np.unique(data[:, 0]).tolist()
print(frames[0])
print(len(frames))

1.0
727


In [42]:
frame_data = []
# data를 frame 단위로 변환
for frame in frames:
    frame_data.append(data[frame == data[:, 0], :]) # 전체 frame에(7994개 scene) 대해, 각 frame별 frame_data (agent 정보, x, y) --> 2차원 list

In [43]:
print(len(frame_data))
print(type(frame_data))
print(frame_data[0])
print(len(frame_data[0]))

727
<class 'list'>
[[ 1.00000000e+00  1.00000000e+00  5.38830520e+05  4.81401216e+06
   6.11800000e+01 -2.24500000e-01  1.16600000e-01  5.29890000e+00
   0.00000000e+00  0.00000000e+00]
 [ 1.00000000e+00  2.00000000e+00  5.38801510e+05  4.81398483e+06
   2.14122000e+01  4.90000000e-02  4.00000000e-03  4.34800000e-01
   0.00000000e+00  0.00000000e+00]
 [ 1.00000000e+00  3.00000000e+00  5.38839930e+05  4.81399724e+06
   5.31595000e+01 -1.74100000e-01  2.60000000e-03  5.30940000e+00
   0.00000000e+00  0.00000000e+00]
 [ 1.00000000e+00  4.00000000e+00  5.38853070e+05  4.81402927e+06
   1.50000000e-03  1.50000000e-03 -1.30000000e-03  3.59490000e+00
   0.00000000e+00  0.00000000e+00]
 [ 1.00000000e+00  5.00000000e+00  5.38807070e+05  4.81399478e+06
   2.63638000e+01  1.08700000e-01 -8.30000000e-03  4.46300000e-01
   0.00000000e+00  0.00000000e+00]
 [ 1.00000000e+00  6.00000000e+00  5.38851070e+05  4.81402437e+06
   3.90000000e-03  2.80000000e-03  5.80000000e-03  3.66580000e+00
   0.00000000e

In [17]:
print(frame_data[0:20])
print(len(frame_data[0:20]))


[array([[ 1.00000000e+00,  1.00000000e+00,  5.38830520e+05,
         4.81401216e+06,  6.11800000e+01, -2.24500000e-01,
         1.16600000e-01,  5.29890000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 1.00000000e+00,  2.00000000e+00,  5.38801510e+05,
         4.81398483e+06,  2.14122000e+01,  4.90000000e-02,
         4.00000000e-03,  4.34800000e-01,  0.00000000e+00,
         0.00000000e+00],
       [ 1.00000000e+00,  3.00000000e+00,  5.38839930e+05,
         4.81399724e+06,  5.31595000e+01, -1.74100000e-01,
         2.60000000e-03,  5.30940000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 1.00000000e+00,  4.00000000e+00,  5.38853070e+05,
         4.81402927e+06,  1.50000000e-03,  1.50000000e-03,
        -1.30000000e-03,  3.59490000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 1.00000000e+00,  5.00000000e+00,  5.38807070e+05,
         4.81399478e+06,  2.63638000e+01,  1.08700000e-01,
        -8.30000000e-03,  4.46300000e-01,  0.00000000e+00,
         0

In [12]:
seq_len=20
skip=1

num_sequences = int(math.ceil((len(frames) - seq_len + 1) / skip)) # seqence 개수
print(num_sequences)

708


In [13]:
obs_len = 8
pred_len = 12
threshold = 0.002
min_agent =1 
num_agents_in_seq = []
seq_list = []
non_linear_agent = []
loss_mask_list = []
seq_list_rel = []

for idx in range(0, num_sequences * skip + 1, skip):
    curr_seq_data = np.concatenate(                         # num_sequnce만큼 끊어서 curr seq_data 생성
        frame_data[idx:idx + seq_len], axis=0               # ex) seq 1: 1-20 frame, seq 2: 2-21 frame ... 
    )
    # print(idx) # 7976 (sequnece 개수)
    # print(curr_seq_data)
    # print(len(curr_seq_data)) # 현재 시퀀스에 있는 row (프레임 별 agent)의 합 --> 20개 프레임에 대한 agent (중복 포함) 합 --> 20*20= 400

    agents_in_curr_seq = np.unique(curr_seq_data[:, 1])       # 현재 seq에 있는 agents 목록, 모든 행의 1번째(agent 정보) 열 slicing
    # print(len(agents_in_curr_seq))
    # print(agents_in_curr_seq)
    curr_seq_rel = np.zeros((len(agents_in_curr_seq), 2,        # (현재 seq에 있는 agents 개수, 2, seq_len) 
                                seq_len))
    # print(curr_seq_rel)
    curr_seq = np.zeros((len(agents_in_curr_seq), 2, seq_len))   # (현재 seq에 있는 agents 개수, 2, seq_len) 
    curr_loss_mask = np.zeros((len(agents_in_curr_seq),
                                seq_len))
    num_agents_considered = 0
    _non_linear_agent = []
    for _, agent_id in enumerate(agents_in_curr_seq):           # 현재 seq에 있는 agents 목록
        curr_agent_seq = curr_seq_data[ curr_seq_data[:, 1] == # 현재 seq에 있는 agent의 seqeunce
                                        agent_id, :]
        # print(curr_agent_seq)
        # print(agent_id)
        curr_agent_seq = np.around(curr_agent_seq, decimals=4)
        pad_front = frames.index(curr_agent_seq[0, 0]) - idx
        pad_end = frames.index(curr_agent_seq[-1, 0]) - idx + 1
        if pad_end - pad_front != seq_len:
            continue
        curr_agent_seq = np.transpose(curr_agent_seq[:, 2:])
        curr_agent_seq = curr_agent_seq
        # Make coordinates relative
        rel_curr_agent_seq = np.zeros(curr_agent_seq.shape)
        rel_curr_agent_seq[:, 1:] = \
            curr_agent_seq[:, 1:] - curr_agent_seq[:, :-1]
        _idx = num_agents_considered
        curr_seq[_idx, :, pad_front:pad_end] = curr_agent_seq
        curr_seq_rel[_idx, :, pad_front:pad_end] = rel_curr_agent_seq
        # Linear vs Non-Linear Trajectory
        _non_linear_agent.append(
            poly_fit(curr_agent_seq, pred_len 
                        ,threshold
                        )
                        )
        curr_loss_mask[_idx, pad_front:pad_end] = 1
        num_agents_considered += 1
        
    if num_agents_considered > min_agent:
        non_linear_agent += _non_linear_agent
        num_agents_in_seq.append(num_agents_considered)
        loss_mask_list.append(curr_loss_mask[:num_agents_considered])
        seq_list.append(curr_seq[:num_agents_considered])
        seq_list_rel.append(curr_seq_rel[:num_agents_considered])

num_seq = len(seq_list)
        
seq_list = np.concatenate(seq_list, axis=0)
seq_list_rel = np.concatenate(seq_list_rel, axis=0)

loss_mask_list = np.concatenate(loss_mask_list, axis=0)
non_linear_agent = np.asarray(non_linear_agent)

        # Convert numpy -> Torch Tensor
obs_traj = torch.from_numpy(
    seq_list[:, :, :obs_len]).type(torch.float)
pred_traj = torch.from_numpy(
    seq_list[:, :, obs_len:]).type(torch.float)

obs_traj_rel = torch.from_numpy(
    seq_list_rel[:, :, :obs_len]).type(torch.float)
pred_traj_rel = torch.from_numpy(
    seq_list_rel[:, :, obs_len:]).type(torch.float)

loss_mask = torch.from_numpy(loss_mask_list).type(torch.float)
non_linear_agent = torch.from_numpy(non_linear_agent).type(torch.float)
cum_start_idx = [0] + np.cumsum(num_agents_in_seq).tolist()
seq_start_end = [
    (start, end)
    for start, end in zip(cum_start_idx, cum_start_idx[1:])
]

def len():              # len(train_dset)
    return num_seq

def getitem(index):
    start, end = seq_start_end[index]
    out = [
        obs_traj[start:end, :], 
        pred_traj[start:end, :],
        obs_traj_rel[start:end, :], 
        pred_traj_rel[start:end, :],
        non_linear_agent[start:end], 
        loss_mask[start:end, :]
    ]
    return out

ValueError: could not broadcast input array from shape (8,20) into shape (2,20)

In [63]:
print(curr_seq_data)
print()
print(len(curr_seq_data))
print()
print(agents_in_curr_seq)

print()
print(len(agents_in_curr_seq))
print()

[[8.70100000e+03 2.03000000e+02 5.38825620e+05 4.81400567e+06]
 [8.70100000e+03 2.05000000e+02 5.38825260e+05 4.81400510e+06]
 [8.70100000e+03 2.34000000e+02 5.38841880e+05 4.81400071e+06]
 ...
 [8.72000000e+03 2.80000000e+02 5.38846450e+05 4.81398814e+06]
 [8.72000000e+03 2.82000000e+02 5.38832150e+05 4.81402206e+06]
 [8.72000000e+03 2.83000000e+02 5.38838200e+05 4.81401869e+06]]

380

[203. 205. 234. 249. 256. 257. 258. 259. 260. 261. 262. 270. 273. 275.
 277. 278. 279. 280. 282. 283.]

20

